### import module_structuration_revues

In [18]:
import pandas as pd
import module_structuration_revues
# import ESSAI_module_structuration_revues

# AHMC

In [19]:
with open ("Liste-URL-AHMC.txt","r") as Liste_URL_AHMC :
    liste_url_AHMC = Liste_URL_AHMC.read().split(" ; ")

In [20]:
l = []
df = pd.DataFrame()
for url in liste_url_AHMC:
    df2 =  module_structuration_revues.url_to_df_AHMC(url)
    df = pd.concat([df,df2])

In [21]:
df.to_csv('AHMC_articles.csv')

In [22]:
df

,article_id,article_auteur,article_detail,article_titre,revue_annee,revue_numero,article_page
0,AHMC-01-9,le Dr Clarac,Première partie,Contribution à la géographie médicale. Notes ...,1898,01,9
1,AHMC-01-43,le Dr Clarac. Première partie. Deuxième partie,La fièvre hémoglobinurique endémique observée...,Contribution à la géographie médicale. Notes ...,1898,01,43
2,AHMC-01-114,le Dr Fontaine(1),NaN,Notes sur la mortalité des troupes d'infanteri...,1898,01,114
3,AHMC-01-129,le Dr Calmette,NaN,Sur le mécanisme de l'immunisation contre les ...,1898,01,129
4,AHMC-01-135,NaN,NaN,Notes succinctes sur l'état sanitaire de nos ...,1898,01,135
...,...,...,...,...,...,...,...
44,AHMC-38-351,MM. Les Drs Grosfilez et Lefèvre. Chapitre IV....,Cancer,II. Chronique documentaire. Les maladies tran...,1940,38,351
45,AHMC-38-355,MM. Les Drs Grosfilez et Lefèvre. Chapitre IV....,Alcoolisme,II. Chronique documentaire. Les maladies tran...,1940,38,355
46,AHMC-38-360,NaN,NaN,II. Chronique documentaire. Commission de la ...,1940,38,360
47,AHMC-38-382,NaN,NaN,III. Promotions. Nominations,1940,38,382


In [14]:
df_sans_doublons = module_structuration_revues.df_sans_doublons(df)

In [17]:
df_sans_doublons.to_csv("~/Bureau/Structuration-donnees/Revues/AHMC/AHMC_nodoublons.csv")

# AMN 

In [2]:
with open ("Liste-URL-AMN.txt","r") as Liste_URL_AMN :
    liste_url_AMN = Liste_URL_AMN.read().split(" ; ")
    #print(liste_url_AMN)

In [3]:
l = []
df = pd.DataFrame()
for url in liste_url_AMN:
    df2 = module_structuration_revues.url_to_df_AMN(url)
    df = pd.concat([df,df2])

UnboundLocalError: local variable 'revue_annee' referenced before assignment

In [ ]:
print(df)

In [5]:
from bs4 import BeautifulSoup
from lxml import html
import requests
import regex as re
import pandas as pd

r = requests.get("https://www.biusante.parisdescartes.fr/histoire/medica/resultats/index.php?do=chapitre&cote=90156x1897x67")

##### extraction du corpus html du numéro 
numero_AMN_soup = BeautifulSoup (r.text, features="lxml")

##### extration des balises <div class='titre' ...> </div>
div_ens_tag = numero_AMN_soup.find_all("div", {"class":"tableau1"})


##### 1.2.2. extraire les href / l'année / le numéro
for balise in div_ens_tag:
    l = list(balise)
    ahref = str(l[1])
    intermed = re.search(r"x([0-9]{4})x([0-9]{2})",ahref)
    intermed = str(intermed)

    #isoler année : 
    if re.search(r"[0-9]{4}",intermed) is None:
        pass
    else :
        revue_annee = re.search(r"[0-9]{4}",intermed).group(0)

    #isoler numéro : 
    if re.search(r"[0-9]{4}",intermed) is None:
        pass
    else :
        revue_numero = re.search(r"([0-9]{2})'",intermed).group(1)


#### 2 : Nettoyer les entrées pour ne conserver que les articles#####

#### 2.1. Liste ne conservant que la page et le contenu de l'entrée

liste_page_entrees = []
Liste_entrees = []

######### NB : je fais exprès de dire "entrée" et non "article" car il y a d'autres contenus que des articles
for balise in div_ens_tag:
    liste_page_entrees.append(balise.text)

for entree in liste_page_entrees :
    RE_entree = re.findall(r"\t(\d{1,3})(.*)",entree)
    Liste_entrees.append(RE_entree)

#print(Liste_entrees)
#### 2.2. Pour les pages d'un nouvel article qui débute, ne conserver que le nom du nouveau
Liste_entrees_no_doublesentrees= []

for entree_t_list in Liste_entrees :
    for entree_t_tupple in entree_t_list :
            tupple_to_list = list(entree_t_tupple)
            tupple_to_list.append(revue_annee)
            #type(revue_annee))= str
            tupple_to_list.append(revue_numero)